In [26]:
import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.models import load_model

In [38]:
# Load models
with open('one_hot_encoder_Gender.pkl','rb') as file:
	one_hot_encoder_Gender=pickle.load(file)

with open('one_hot_encoder_Geography.pkl','rb') as file:
	one_hot_encoder_Geography=pickle.load(file)

with open('scalar.pkl','rb') as file:
	scalar=pickle.load(file)

model=load_model("model.h5")

In [28]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [29]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [30]:
one_hot_encoder_Gender.get_feature_names_out

<bound method OneHotEncoder.get_feature_names_out of OneHotEncoder()>

In [31]:
gender_encoder=one_hot_encoder_Gender.transform(input_df[['Gender']]).toarray()
geography_encoder=one_hot_encoder_Geography.transform(input_df[['Geography']]).toarray()

In [32]:
input_df_encoded=pd.concat([
	input_df.drop(columns=['Gender','Geography'],axis=1),
	pd.DataFrame(gender_encoder,columns=one_hot_encoder_Gender.get_feature_names_out()),
	pd.DataFrame(geography_encoder,columns=one_hot_encoder_Geography.get_feature_names_out())

],axis=1)

input_df_encoded

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Female,Gender_Male,Geography_France,Geography_Germany,Geography_Spain
0,600,40,3,60000,2,1,1,50000,0.0,1.0,1.0,0.0,0.0


In [42]:
input_df_scaled=scalar.transform(input_df_encoded)
input_df_scaled

array([[-0.51735625,  0.10052768, -0.69356144, -0.26765738,  0.80289406,
         0.6521214 ,  0.97543701, -0.86281594, -0.90160605,  0.90160605,
         1.00627053, -0.57868692, -0.58082558]])

In [46]:
prediction=model.predict(input_df_scaled)
probaliblity_srcore=prediction[0][0]
probaliblity_srcore

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


0.031658605

In [47]:
if probaliblity_srcore > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
